# Coursera IBM Applied Data Science Capstone 

### Week 5 Final Report :Open a Supermarket in Greater Manchester
### Ziyang Liu

#### 1. import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import geocoder

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


#### 2. Get data from wiki and convert into dataframe

In [2]:
my_url = 'https://en.wikipedia.org/wiki/Category:Areas_of_Greater_Manchester' 
my_data = requests.get(my_url).text # send request

In [3]:
soup = BeautifulSoup(my_data, 'html')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[1].findAll("a"):
    neighborhoodList.append(row.text)

In [4]:
# create a new DataFrame from the list
df1 = pd.DataFrame({"Neighborhood": neighborhoodList})
df1.head()

,Neighborhood
0,Adswood
1,Agecroft
2,Alkrington
3,Ashton upon Mersey
4,"Bedford, Greater Manchester"


In [5]:
# There should be 81 areas according to wiki
print(df1.shape)
df1.head(10)

(81, 1)


,Neighborhood
0,Adswood
1,Agecroft
2,Alkrington
3,Ashton upon Mersey
4,"Bedford, Greater Manchester"
5,Besses o' th' Barn
6,"Birch, Greater Manchester"
7,Bolholt
8,Boothstown
9,"Bowdon, Greater Manchester"


#### 3. Grab the coordinates

In [27]:
coords = [geocoder.arcgis(neighborhood).latlng for neighborhood in df1["Neighborhood"].tolist() ]

In [55]:
coords[44] = geocoder.arcgis("Bolholt, Greater Manchester").latlng

In [56]:
coords

[[53.515143548000026, -2.2842399979999755],
 [53.51667000000003, -2.2999999999999545],
 [51.685110000000066, -2.4401999999999475],
 [53.43209005120774, -2.316314928380919],
 [53.42909263972213, -2.4536926368791456],
 [53.515143548000026, -2.2842399979999755],
 [53.45160533703289, -2.2154790442075956],
 [53.515143548000026, -2.2842399979999755],
 [53.502530000000036, -2.4163599999999406],
 [53.515143548000026, -2.2842399979999755],
 [53.515143548000026, -2.2842399979999755],
 [53.35742000000005, -2.166209999999978],
 [53.60890000000006, -2.31356999999997],
 [53.515143548000026, -2.2842399979999755],
 [53.614720000000034, -2.410829999999976],
 [53.515143548000026, -2.2842399979999755],
 [53.515143548000026, -2.2842399979999755],
 [53.50652000000008, -2.0176799999999275],
 [53.52030519337831, -2.355570075476416],
 [53.515143548000026, -2.2842399979999755],
 [53.515143548000026, -2.2842399979999755],
 [52.35918000000004, -2.365099999999927],
 [53.45792000000006, -2.355289999999968],
 [53.6

In [57]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [58]:
df1['Latitude'] = df_coords['Latitude']
df1['Longitude'] = df_coords['Longitude']
print(df1.shape)
df1.head(10)

(81, 3)


,Neighborhood,Latitude,Longitude
0,Adswood,53.515144,-2.284240
1,Agecroft,53.516670,-2.300000
2,Alkrington,51.685110,-2.440200
3,Ashton upon Mersey,53.432090,-2.316315
4,"Bedford, Greater Manchester",53.429093,-2.453693
5,Besses o' th' Barn,53.515144,-2.284240
6,"Birch, Greater Manchester",53.451605,-2.215479
7,Bolholt,53.515144,-2.284240
8,Boothstown,53.502530,-2.416360
9,"Bowdon, Greater Manchester",53.515144,-2.284240


In [59]:
# save the DataFrame as CSV file
df1.to_csv("df1.csv", index=False)

#### 4. create a map 

In [62]:
gman_lat = 53.4576
gman_lng = -2.1578

# Creates map of Greater Manchester using latitude and longitude values
map_man = folium.Map(location=[gman_lat, gman_lng], zoom_start=10)

# Add markers to map
for lat, lng, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_man)  
    
map_man

#### 5. use API to expolre thoes neighborhoods

In [121]:
# Define Foursquare Credentials and Version
CLIENT_ID = '5VFSSSK4X2QIWZNYO3LWRU4YFIQN3TWF0CRFH5PNTVTLD324' # your Foursquare ID
CLIENT_SECRET = 'XTZQD2HWAURYL4JEZRPKYFBWTW0AL0JJ5D411RTTQOBAGNCL' # your Foursquare Secret
VERSION = '20200705' # Foursquare API version
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 5VFSSSK4X2QIWZNYO3LWRU4YFIQN3TWF0CRFH5PNTVTLD324
CLIENT_SECRET:XTZQD2HWAURYL4JEZRPKYFBWTW0AL0JJ5D411RTTQOBAGNCL


In [122]:
# let's get the top 100 venues that are in Toronto a radius of 500 meters.
radius = 500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [123]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(359, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adswood,53.515144,-2.284240,Brian Milligan Associates,53.515355,-2.279747,Construction & Landscaping
1,Adswood,53.515144,-2.284240,Preswich Golf Club,53.518877,-2.283238,Golf Course
2,Agecroft,53.516670,-2.300000,IT Lab,53.514363,-2.293782,IT Services
3,Ashton upon Mersey,53.432090,-2.316315,The Bridge Inn,53.430615,-2.313137,Pub
4,Ashton upon Mersey,53.432090,-2.316315,Majestic Wine,53.430823,-2.318874,Wine Shop


In [124]:
venues_df['VenueCategory'].unique()
# We seee that Shopping mall is included in the VenueCategory

array(['Construction & Landscaping', 'Golf Course', 'IT Services', 'Pub',
       'Wine Shop', 'American Restaurant', 'Track', 'Café',
       'Tram Station', 'Sporting Goods Shop', 'Flower Shop', 'Park',
       'Train Station', 'Hotel', 'Pastry Shop', 'Persian Restaurant',
       'Turkish Restaurant', 'Music Venue', 'Bed & Breakfast',
       'Art Museum', 'Food Truck', 'Supermarket', 'Coffee Shop',
       'Restaurant', 'Harbor / Marina', 'Bakery', 'Bar',
       'Indian Restaurant', 'Pharmacy', 'Mediterranean Restaurant',
       'Italian Restaurant', 'Grocery Store', 'Pizza Place', 'Bookstore',
       'Playground', 'Chinese Restaurant', 'Bus Stop', 'Dog Run',
       'Furniture / Home Store', 'Food', 'Convenience Store', 'Bistro',
       'Burger Joint', 'Other Great Outdoors', 'Rental Car Location',
       'Rugby Stadium', 'Soccer Stadium', 'Vegetarian / Vegan Restaurant',
       'Deli / Bodega', 'Pet Store', 'Climbing Gym', 'Clothing Store',
       'Discount Store', 'Outlet Store', 'Ligh

#### 6.analyze neighborhood

In [125]:
# one hot encoding
manchester_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manchester_onehot['Neighborhood'] = venues_df['Neighborhood'] 

print(manchester_onehot.shape)
manchester_onehot.head()

(359, 96)


,Afghan Restaurant,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Bistro,Bookstore,Burger Joint,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Lake,Light Rail Station,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,Other Great Outdoors,Outdoor Supply Store,Outlet Store,Paintball Field,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Rugby Stadium,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Theater,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Shop,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adswood
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Adswood
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Agecroft
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ashton upon Mersey
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Ashton upon Mersey


In [126]:
manchester_onehot['Neighborhood']

0                              Adswood
1                              Adswood
2                             Agecroft
3                   Ashton upon Mersey
4                   Ashton upon Mersey
5                   Ashton upon Mersey
6                   Ashton upon Mersey
7                   Ashton upon Mersey
8                   Ashton upon Mersey
9                   Ashton upon Mersey
10                  Ashton upon Mersey
11                  Ashton upon Mersey
12         Bedford, Greater Manchester
13         Bedford, Greater Manchester
14         Bedford, Greater Manchester
15         Bedford, Greater Manchester
16                  Besses o' th' Barn
17                  Besses o' th' Barn
18           Birch, Greater Manchester
19           Birch, Greater Manchester
20           Birch, Greater Manchester
21           Birch, Greater Manchester
22           Birch, Greater Manchester
23           Birch, Greater Manchester
24           Birch, Greater Manchester
25           Birch, Great

In [127]:
# move neighborhood column to the first column
fixed_columns = [manchester_onehot.columns[-1]] + list(manchester_onehot.columns[:-1])
manchester_onehot = manchester_onehot[fixed_columns]

print(manchester_onehot.shape)
manchester_onehot

(359, 96)


,Neighborhood,Afghan Restaurant,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Bistro,Bookstore,Burger Joint,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Lake,Light Rail Station,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,Other Great Outdoors,Outdoor Supply Store,Outlet Store,Paintball Field,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Rugby Stadium,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Theater,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Shop
0,Adswood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Adswood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Agecroft,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ashton upon Mersey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ashton upon Mersey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
5,Ashton upon Mersey,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Ashton upon Mersey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7,Ashton upon Mersey,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Ashton upon Mersey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9,Ashton upon Mersey,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [128]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
manchester_grouped = manchester_onehot.groupby('Neighborhood').mean().reset_index()
manchester_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Art Museum,Asian Restaurant,Bakery,Bar,Bed & Breakfast,Beer Bar,Bistro,Bookstore,Burger Joint,Bus Stop,Café,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Dog Run,Falafel Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gas Station,Gastropub,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Home Service,Hookah Bar,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Lake,Light Rail Station,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Movie Theater,Music Venue,Other Great Outdoors,Outdoor Supply Store,Outlet Store,Paintball Field,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Playground,Portuguese Restaurant,Pub,Rental Car Location,Restaurant,Rugby Stadium,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Soccer Field,Soccer Stadium,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Supermarket,Theater,Track,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Shop
0,Adswood,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.00,0.0000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.50,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.5,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000,0.0,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
1,Agecroft,0.000,0.000000,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.00,0.0000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000,0.000000,1.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000,0.0,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000
2,Ashton upon Mersey,0.000,0.111111,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.00,0.0000,0.00,0.0,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000,0.000000,0.000000,0.111111,0.00,0.0,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000,0.0,0.00,0.000000,0.0,0.111111,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.111111,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.111111,0.00,0.0,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.111111,0.0000,0.000000,0.111111
3,"Bedford, Greater Manchester",0.000,0.250000,0.000,0.000,0.000000,0.000000,0.000,0.000000,0.00,0.0000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.00,0.000,0.250000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000,0.0,0.00,0.000000,0.0,0.000000,0.000,0.000000,0.000,0.000000,0.000000,

#### 7. find Shopping mall frequency

In [143]:
manchester_mall = manchester_grouped[["Neighborhood","Shopping Mall"]]
manchester_mall

,Neighborhood,Shopping Mall
0,Adswood,0.000000
1,Agecroft,0.000000
2,Ashton upon Mersey,0.000000
3,"Bedford, Greater Manchester",0.000000
4,Besses o' th' Barn,0.000000
5,"Birch, Greater Manchester",0.000000
6,Bolholt,0.000000
7,Boothstown,0.000000
8,"Bowdon, Greater Manchester",0.000000
9,"Bradshaw, Greater Manchester",0.000000


In [144]:
# Some neighborhoods are missing
df3 = pd.DataFrame({'Neighborhood':['Alkrington', 'Hattersley', 'Higher End', 'Wingates'],
                    'Shopping Mall':[0.000000,0.000000,0.000000,0.000000]})
manchester_mall = pd.concat([manchester_mall, df3],axis=0,ignore_index=True)
manchester_mall.head()

,Neighborhood,Shopping Mall
0,Adswood,0.0
1,Agecroft,0.0
2,Ashton upon Mersey,0.0
3,"Bedford, Greater Manchester",0.0
4,Besses o' th' Barn,0.0


In [153]:
manchester_mall.sort_values(['Neighborhood'], inplace = True)

In [152]:
manchester_mall.reset_index(drop = True, inplace = True)
manchester_mall.head()

,Neighborhood,Shopping Mall
0,Adswood,0.0
1,Agecroft,0.0
2,Alkrington,0.0
3,Ashton upon Mersey,0.0
4,"Bedford, Greater Manchester",0.0


#### 8. Clustering

In [154]:
# set number of clusters
kclusters = 3

manchester_clustering = manchester_mall.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manchester_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

C:\software\Anaconda\lib\site-packages\ipykernel_launcher.py:7: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  import sys


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [155]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
manchester_merged = manchester_mall.copy()

# add clustering labels
manchester_merged["Cluster Labels"] = kmeans.labels_
manchester_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
manchester_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Adswood,0.0,0
1,Agecroft,0.0,0
2,Alkrington,0.0,0
3,Ashton upon Mersey,0.0,0
4,"Bedford, Greater Manchester",0.0,0


In [156]:
manchester_merged[manchester_merged['Cluster Labels']==1]

,Neighborhood,Shopping Mall,Cluster Labels
74,Urmston,0.045455,1


In [157]:
manchester_merged[manchester_merged['Cluster Labels']==2]

,Neighborhood,Shopping Mall,Cluster Labels


In [158]:
manchester_merged[manchester_merged['Cluster Labels']==0]

,Neighborhood,Shopping Mall,Cluster Labels
0,Adswood,0.0,0
1,Agecroft,0.0,0
2,Alkrington,0.0,0
3,Ashton upon Mersey,0.0,0
4,"Bedford, Greater Manchester",0.0,0
5,Besses o' th' Barn,0.0,0
6,"Birch, Greater Manchester",0.0,0
7,Bolholt,0.0,0
8,Boothstown,0.0,0
9,"Bowdon, Greater Manchester",0.0,0


In [164]:
"Urmston is the best place to open a shopping mall."

'Urmston is the best place to open a shopping mall.'

In [161]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manchester_merged = manchester_merged.join(df1.set_index("Neighborhood"), on="Neighborhood")

print(manchester_merged.shape)
manchester_merged.head() # check the last columns!

(81, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Adswood,0.0,0,53.515144,-2.284240
1,Agecroft,0.0,0,53.516670,-2.300000
2,Alkrington,0.0,0,51.685110,-2.440200
3,Ashton upon Mersey,0.0,0,53.432090,-2.316315
4,"Bedford, Greater Manchester",0.0,0,53.429093,-2.453693


#### 8. Visualizing Clusters

In [163]:
# create map
map_clusters = folium.Map(location=[gman_lat, gman_lng], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manchester_merged['Latitude'], manchester_merged['Longitude'], manchester_merged['Neighborhood'], manchester_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters